In [1]:
from itertools import chain
from random import random

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from society.action import Action
from society.agent import Agent, TrainableAgent
from society.simulations.weighted import WeightedNetworkSimulation
from society.strategies.gameplay.constant import AllC, AllD
from society.strategies.gameplay.qlearning import TabularQLearningGameplayStrategy
from society.strategies.gameplay.random import RandomGameplayStrategy
from society.strategies.gameplay.tft import TitForTat
from society.visualisation.network import *

plt.rcParams["figure.figsize"] = (10, 6)

In [2]:
def generate_population(communities, size):
    population = communities * size

    agents = [
        Agent(TabularQLearningGameplayStrategy(lookback=4, epsilon=0.1), i, population)
        for i in range(population)
    ]

    G = nx.connected_caveman_graph(communities, size)

    weights_matrix = np.zeros((population, population))
    for u, v, d in G.edges(data=True):
        weights_matrix[u, v] = weights_matrix[v, u] = 1.0
        try:
            d["weight"] = weights_matrix[u, v]
        except:
            d["weight"] = 0

    return agents, weights_matrix, G

In [3]:
def compute_policies(agents):
    policies = [
        agent.gameplay_strategy._q_table.argmax(axis=-1)
        for agent in agents
    ]

    return [
        "".join(
            [
                ("C", "D")[policy[i, j, k, l]]
                for i in range(4)
                for j in range(4)
                for k in range(4)
                for l in range(4)
            ]
        )
        for policy in policies
    ]

def calculate_cooperativeness(history):
    count = history.count(Action.COOPERATE)

    return count / len(history)

In [4]:
# PAYOFF_LABELS = ["(C, C)", "(C, D)", "(D, C)", "(D, D)"]

PAYOFF_LABELS = ["R", "S", "T", "P"]

POPULATION = (2, 32)
ROUNDS = 25_000

run_rewards = []
run_cumulative_rewards = []
run_action_histories = []
run_reward_histories = []
run_policies = []

for run in range(20):
    # Generate a new population
    agents, weights_matrix, G = generate_population(*POPULATION)

    # Run a number of rounds
    sim = WeightedNetworkSimulation(agents, weights_matrix)

    # Run the simulation
    sim.reset()
    for i in tqdm(range(ROUNDS), desc=f"Run {run + 1}"):
        sim.play_round(train=True)

    # Store results
    run_rewards.append(sim.rewards)
    run_action_histories.append(sim.action_histories)
    run_reward_histories.append(sim.reward_histories)
    run_policies.append(compute_policies(agents))

    # Output statistics for the best and worst agents
    cumulative_rewards = [(i, sum(history)) for i, history in enumerate(sim.reward_histories)]
    cumulative_rewards.sort(key=lambda x: x[1], reverse=True)

    tqdm.write(
        f"BEST AGENT: {cumulative_rewards[0][1]} ({calculate_cooperativeness(list(chain(*sim.action_histories[cumulative_rewards[0][0]])))}, {run_policies[-1][cumulative_rewards[0][0]]})"
    )
    tqdm.write(
        f"WORST AGENT: {cumulative_rewards[-1][1]} ({calculate_cooperativeness(list(chain(*sim.action_histories[cumulative_rewards[-1][0]])))}, {run_policies[-1][cumulative_rewards[-1][0]]})"
    )


Run 1:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 151295 (0.8598968394638838, CDDCDCCCCCCCCCCCCCCCCCDCDCCDCCCCCCCCCCCCCCDCCCCCCCCDCCCCDCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC)
WORST AGENT: 143092 (0.9479669262647901, CCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC)


Run 2:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 162652 (0.7173804232486436, CCCCCCCCCDCDCCCCDCCCCDCCCCCCCCCCCCCDCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCDCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC)
WORST AGENT: 140446 (0.9448769283928535, CCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCDCCCCDCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCDCDCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC)


Run 3:   0%|          | 0/25000 [00:00<?, ?it/s]

...

BEST AGENT: 162769 (0.7233587725256723, CCCCCCCCCCCCCDCCDCDCCCCDCCCCCCCCCCCCDCCDCCCDCCCCDCCCCDCCCCCDCCCCCDCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCDCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCC)
WORST AGENT: 138445 (0.9455918990597123, CCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCDCCCCCCCCCCCCCCDCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCDCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC)


Run 19:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 162898 (0.7205616298277354, DCCCCCCDCCCDCCCCCCCCCCCCCCCDCCCCDDCCCCDCCCCCCCCCCCCDCCCCCCCCCCCCCCCCDCCCCDCCDCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCDCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC)
WORST AGENT: 139800 (0.9399995937684074, CCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCDCCCCCCCCCCCCCCDDCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCDCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC)


Run 20:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 153190 (0.8495302173739947, CDDCCCDCCCCCDCCCCCCCCDCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDDCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCDDCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCC)
WORST AGENT: 143109 (0.9073300573300573, CCCCDCCCCCCDCCCCCCCCCCDCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDDCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCC)


In [5]:
from datetime import datetime
import pickle

with open(f"results - {datetime.isoformat(datetime.now()).replace(':', '-')}.pickle", "wb") as f:
    pickle.dump({
        "rewards": run_rewards,
        "cumulative_rewards": run_cumulative_rewards,
        # "action_histories": run_action_histories,
        "reward_histories": run_reward_histories,
        "policies": run_policies,
    }, f)


In [6]:
cumulative_reward_policy_ranks = {}
cumulative_reward_policies = {}

for policies, reward_histories in zip(run_policies, run_reward_histories):
    cumulative_rewards = [(i, sum(history)) for i, history in enumerate(reward_histories)]
    cumulative_rewards.sort(key=lambda x: x[1], reverse=True)

    for rank, (i, cumulative_reward) in enumerate(cumulative_rewards):
        policy = policies[i]

        if policy not in cumulative_reward_policy_ranks:
            cumulative_reward_policy_ranks[policy] = []
            cumulative_reward_policies[policy] = []

        cumulative_reward_policy_ranks[policy].append(rank + 1)
        cumulative_reward_policies[policy].append(cumulative_reward)

mean_cumulative_reward_policy_ranks = {policy: np.mean(ranks) for policy, ranks in cumulative_reward_policy_ranks.items()}
mean_cumulative_reward_policies = {policy: np.mean(rewards) for policy, rewards in cumulative_reward_policies.items()}

In [7]:
for policy in sorted(mean_cumulative_reward_policy_ranks, key=lambda x: mean_cumulative_reward_policy_ranks[x]):
    print(f"{mean_cumulative_reward_policy_ranks[policy]:<24} {policy}")

1.0                      CDDCDCCCCCCCCCCCCCCCCCDCDCCDCCCCCCCCCCCCCCDCCCCCCCCDCCCCDCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
1.0                      CCCCCCCCCDCDCCCCDCCCCDCCCCCCCCCCCCCDCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCDCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
1.0                      CDCCDCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCDCDCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCDCCCCDCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCDCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
1.0                      CCCCCCCCDDCDCDCCCCCCCCCCCDCCCCCCDCCCCDCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

In [8]:
for policy in sorted(mean_cumulative_reward_policies, key=lambda x: mean_cumulative_reward_policies[x], reverse=True):
    print(f"{mean_cumulative_reward_policies[policy]:<24} {policy}")

170692.0                 DDDCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCDCCDCCCCCCCCCCCCCCDCDCCCDCCCCCCCCCCCDCCCCCCCCCCCCCCCCCDCCCDCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCDCCCCCDCCCCCCCCCCDDCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
165322.0                 DCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCDCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCDCCCDDDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
164625.0                 DCCCCDCCDCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCDCCCCCCCCCCCCDDCCCCCDCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDDCCCDCCDCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
164488.0                 DCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCCCCCCDCCDCCCCCCCCCCCCCCCCCCCCCCDCDCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCDCCCD

In [14]:
compute_policies([agents[cumulative_rewards[0][0]]])

['CDDCCCDCCCCCDCCCCCCCCDCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDDCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCDDCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCC']

In [13]:
agents[cumulative_rewards[0][0]].gameplay_strategy._q_table

array([[[[[2.18813805e+02, 2.01637681e+02],
          [1.87651357e+02, 2.05583682e+02],
          [1.86583548e+02, 2.00882908e+02],
          [1.37578170e+02, 2.92409767e+00]],

         [[2.05004948e+02, 6.79126166e+01],
          [1.16026239e+02, 2.33551411e+00],
          [1.80460637e+02, 1.98120594e+02],
          [1.40575596e+02, 3.88993283e+00]],

         [[2.08968733e+02, 2.09689489e+01],
          [6.85459491e+01, 0.00000000e+00],
          [2.01901156e+02, 1.30837916e+02],
          [1.64754785e+02, 1.50342921e+01]],

         [[1.08590936e+01, 1.71634965e+02],
          [4.74709509e+01, 9.50000000e-01],
          [5.82102257e+00, 0.00000000e+00],
          [0.00000000e+00, 0.00000000e+00]]],


        [[[2.08997778e+02, 1.72499720e+02],
          [1.77375039e+02, 2.13165564e+01],
          [1.55216926e+02, 5.92302447e+00],
          [1.17864360e+00, 0.00000000e+00]],

         [[1.81105896e+02, 0.00000000e+00],
          [2.97000000e-02, 6.24700000e-01],
          [1.8400533

In [17]:
from itertools import product

P = compute_policies([agents[cumulative_rewards[0][0]]])[0]

for m, p in zip(product(PAYOFF_LABELS, PAYOFF_LABELS, PAYOFF_LABELS, PAYOFF_LABELS), P):
    print(m, "=>", p)

('R', 'R', 'R', 'R') => C
('R', 'R', 'R', 'S') => D
('R', 'R', 'R', 'T') => D
('R', 'R', 'R', 'P') => C
('R', 'R', 'S', 'R') => C
('R', 'R', 'S', 'S') => C
('R', 'R', 'S', 'T') => D
('R', 'R', 'S', 'P') => C
('R', 'R', 'T', 'R') => C
('R', 'R', 'T', 'S') => C
('R', 'R', 'T', 'T') => C
('R', 'R', 'T', 'P') => C
('R', 'R', 'P', 'R') => D
('R', 'R', 'P', 'S') => C
('R', 'R', 'P', 'T') => C
('R', 'R', 'P', 'P') => C
('R', 'S', 'R', 'R') => C
('R', 'S', 'R', 'S') => C
('R', 'S', 'R', 'T') => C
('R', 'S', 'R', 'P') => C
('R', 'S', 'S', 'R') => C
('R', 'S', 'S', 'S') => D
('R', 'S', 'S', 'T') => C
('R', 'S', 'S', 'P') => C
('R', 'S', 'T', 'R') => C
('R', 'S', 'T', 'S') => C
('R', 'S', 'T', 'T') => C
('R', 'S', 'T', 'P') => C
('R', 'S', 'P', 'R') => C
('R', 'S', 'P', 'S') => C
('R', 'S', 'P', 'T') => C
('R', 'S', 'P', 'P') => C
('R', 'T', 'R', 'R') => C
('R', 'T', 'R', 'S') => C
('R', 'T', 'R', 'T') => D
('R', 'T', 'R', 'P') => C
('R', 'T', 'S', 'R') => C
('R', 'T', 'S', 'S') => C
('R', 'T', '